<a href="https://colab.research.google.com/github/ASHIKalip/DataScience/blob/main/alexnet_Mnist_res.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Corrected AlexNet-like model for MNIST
def create_alexnet_mnist():
    model = keras.Sequential([
        layers.Conv2D(64, (5, 5), activation='relu', padding='same', input_shape=(28, 28, 1)),  # Smaller kernel, same padding
        layers.MaxPooling2D((2, 2), strides=2),  # Regular pooling with stride 2
        layers.Conv2D(128, (5, 5), activation='relu', padding='same'),  # Adjusted kernel size and padding
        layers.MaxPooling2D((2, 2), strides=2),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),  # Smaller kernel, same padding
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),  # Smaller kernel, same padding
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), strides=2),  # Keep max pooling stride 2
        layers.Flatten(),
        layers.Dense(1024, activation='relu'),  # Reduced layer size
        layers.Dropout(0.5),
        layers.Dense(1024, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax'),  # Output layer for 10 classes
    ])
    return model

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

# Create and compile the model
model = create_alexnet_mnist()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))


11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/10
1875/1875 [==============================] - 27s 9ms/step - loss: 0.1873 - accuracy: 0.9410 - val_loss: 0.0627 - val_accuracy: 0.9853
Epoch 2/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0659 - accuracy: 0.9825 - val_loss: 0.0388 - val_accuracy: 0.9897
Epoch 3/10
1875/1875 [==============================] - 24s 13ms/step - loss: 0.0547 - accuracy: 0.9858 - val_loss: 0.0546 - val_accuracy: 0.9859
Epoch 4/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0518 - accuracy: 0.9877 - val_loss: 0.0443 - val_accuracy: 0.9883
Epoch 5/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0435 - accuracy: 0.9885 - val_loss: 0.0410 - val_accuracy: 0.9909
Epoch 6/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0438 - accuracy: 0.9896 - val_loss: 0.0456 - val_accuracy: 0.9881
Epoch 7/10
1875/1875 [==============================] - 15s 8ms/

In [12]:
baseline_loss, baseline_accuracy=model.evaluate(x_test, y_test)
print(baseline_loss,baseline_accuracy)

313/313 [==============================] - 2s 8ms/step - loss: 0.0491 - accuracy: 0.9911
0.049107328057289124 0.991100013256073


In [2]:
# If using Colab, install the required package
!pip install -q tensorflow-model-optimization


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 5.6 MB/s eta 0:00:00


In [4]:
import tensorflow_model_optimization as tfmot

# Adding pruning callbacks
pruning_callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    tfmot.sparsity.keras.PruningSummaries(log_dir='log')  # Optional: For visualization
]


# Apply a pruning wrapper to the model
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.2,  # initial sparsity level
        final_sparsity=0.8,    # final sparsity level
        begin_step=0,
        end_step=2000,         # roughly corresponds to 10 epochs
    )
}

# Apply the pruning wrapper to the original model
pruned_model = prune_low_magnitude(create_alexnet_mnist(), **pruning_params)

pruned_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the pruned model
pruned_model.fit(    x_train,    y_train,    epochs=10,    validation_data=(x_test, y_test),    callbacks=pruning_callbacks)

# Strip the pruning wrapper for better model size
final_model = tfmot.sparsity.keras.strip_pruning(pruned_model)



# Evaluate the final (pruned and stripped) model
pruned_loss, pruned_accuracy = final_model.evaluate(x_test, y_test)


Epoch 1/10
   5/1875 [..............................] - ETA: 27s - loss: 2.3244 - accuracy: 0.1125    

1875/1875 [==============================] - 42s 18ms/step - loss: 0.2075 - accuracy: 0.9326 - val_loss: 0.0399 - val_accuracy: 0.9873
Epoch 2/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.0464 - accuracy: 0.9867 - val_loss: 0.0343 - val_accuracy: 0.9897
Epoch 3/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0349 - accuracy: 0.9902 - val_loss: 0.0325 - val_accuracy: 0.9909
Epoch 4/10
1875/1875 [==============================] - 31s 17ms/step - loss: 0.0297 - accuracy: 0.9910 - val_loss: 0.0246 - val_accuracy: 0.9931
Epoch 5/10
1875/1875 [==============================] - 29s 15ms/step - loss: 0.0242 - accuracy: 0.9926 - val_loss: 0.0390 - val_accuracy: 0.9880
Epoch 6/10
1875/1875 [==============================] - 27s 15ms/step - loss: 0.0208 - accuracy: 0.9939 - val_loss: 0.0274 - val_accuracy: 0.9924
Epoch 7/10
1875/1875 [==============================] - 27s 15ms/step - loss: 0.0180 - accuracy: 0.9947 - val_loss: 0.0301 - val_accura

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [8]:
final_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
pruned_loss, pruned_accuracy=final_model.evaluate(x_test, y_test)
print(pruned_loss, pruned_accuracy)

313/313 [==============================] - 2s 4ms/step - loss: 0.0366 - accuracy: 0.9915
0.03657243773341179 0.9915000200271606


In [13]:
import os

# Save the original model
model.save("alexnet_mnist.h5")

# Save the pruned model
final_model.save("pruned_alexnet_mnist.h5")

# Get the size of the models
original_size = os.path.getsize("alexnet_mnist.h5")
pruned_size = os.path.getsize("pruned_alexnet_mnist.h5")

print(f"Baseline accuracy: {baseline_accuracy}, Baseline model size: {original_size / (1024 * 1024):.2f} MB")
print(f"Pruned accuracy: {pruned_accuracy}, Pruned model size: {pruned_size / (1024 * 1024):.2f} MB")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Baseline accuracy: 0.991100013256073, Baseline model size: 41.58 MB
Pruned accuracy: 0.9915000200271606, Pruned model size: 13.88 MB


In [18]:
# Ensure the dataset is of type FLOAT32
x_train_float32 = x_train.astype("float32")

# Representative dataset function for quantization calibration
def representative_dataset_gen():
    for input_value in tf.data.Dataset.from_tensor_slices((x_train_float32[:100])).batch(1):
        yield [input_value]  # Ensure FLOAT32


In [19]:
# Create a TensorFlow Lite converter for the pruned model
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)

# Enable integer quantization with representative dataset
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.representative_dataset = representative_dataset_gen  # Ensure FLOAT32

# Convert the model to TensorFlow Lite
tflite_model = converter.convert()

# Save the TFLite model
with open("pruned_alexnet_mnist.tflite", "wb") as f:
    f.write(tflite_model)


/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [20]:
import tensorflow as tf
import numpy as np
import time

# Load the TFLite model
tflite_model_path = "pruned_alexnet_mnist.tflite"  # Your TFLite model file
interpreter = tf.lite.Interpreter(model_content=open(tflite_model_path, "rb").read())
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

# Create a test input with the expected shape and type
# Adjust the shape and dtype based on your model
input_shape = input_details["shape"]
input_dtype = input_details["dtype"]
test_input = np.random.random(input_shape).astype(input_dtype)  # Random input for latency check

# Measure latency
num_iterations = 100  # Number of iterations for the test
latency_times = []

for _ in range(num_iterations):
    # Set the input tensor
    interpreter.set_tensor(input_details["index"], test_input)

    # Measure the time for a single inference
    start_time = time.time()
    interpreter.invoke()  # Run inference
    end_time = time.time()

    latency_times.append(end_time - start_time)

# Calculate average latency
average_latency = sum(latency_times) / num_iterations

print(f"Average latency for {num_iterations} inferences: {average_latency:.6f} seconds")


Average latency for 100 inferences: 0.003254 seconds


In [22]:
original_size = os.path.getsize("alexnet_mnist.h5")
pruned_size = os.path.getsize("pruned_alexnet_mnist.tflite")
print(f"Baseline model size: {original_size / (1024 * 1024):.2f} MB")
print(f"Pruned model size: {pruned_size / (1024 * 1024):.2f} MB")

Baseline model size: 41.58 MB
Pruned model size: 3.49 MB


In [25]:
import tensorflow as tf
import numpy as np

# Ensure the dataset is of type FLOAT32
x_train_float32 = x_train.astype("float32")  # Convert to numpy with correct dtype

# Define the representative dataset function for calibration
def representative_dataset_gen():
    # Using a slice of x_train as the representative dataset
    for data in tf.data.Dataset.from_tensor_slices(x_train_float32[:100]).batch(1):
        yield [data]  # Yield TensorFlow tensor in FLOAT32


In [26]:
# Create a TensorFlow Lite converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Enable integer quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.representative_dataset = representative_dataset_gen  # Ensure float32 tensors

# Convert the model to TensorFlow Lite
tflite_model = converter.convert()

# Save the quantized TFLite model
with open("quantized_model.tflite", "wb") as f:
    f.write(tflite_model)

print("Quantized model saved successfully.")


/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Quantized model saved successfully.
